In [1]:
import json
import pandas as pd

In [2]:
# Read data from tsv into dataframe
valData = pd.read_csv ('../data/LIAR-PLUS/dataset/tsv/val2.tsv', sep='\t', header=None)


# Add collumn names in the data.
columnNames = ['id', 'json_file_id', 'label', 'claim', 'topics', 'originator', 'title', 'state', 'party','c1', 'c2', 'c3', 'c4', 'c5', 'location', 'justification']
valData.columns = columnNames

In [3]:
# Transform truth values from six classes to two. 'True' and 'False'
def transformTruthValues(dataFrame, trueLabels=[], falseLabels=[]):
    
    trueList  = []
    falseList = []

    for index, row in dataFrame.iterrows():
        if row['label'] in trueLabels:
            dataFrame.at[index, 'label'] = 'true'
            trueList.append(dataFrame.iloc[index].values)
        else:
            dataFrame.at[index, 'label'] = 'false'
            falseList.append(dataFrame.iloc[index].values)

    trueData  = pd.DataFrame(trueList,  columns=columnNames)
    falseData = pd.DataFrame(falseList, columns=columnNames)
    
    return trueData, falseData

In [4]:
# Fix dataframe ids to start from 0
def fixIds(dataFrame):    
    id = 0
    for index, row in dataFrame.iterrows():
        dataFrame.at[index, 'id'] = id
        id += 1
    
    return dataFrame

In [5]:
# Transform truth values from six classes to two. 'True' and 'False'

trueLabels  = ['mostly-true', 'true']
falseLabels = ['pants-fire', 'false', 'barely-true', 'half-true']

trueData, falseData = transformTruthValues(valData, trueLabels, falseLabels)

In [6]:
# Set pandas options show that dataframes appear not truncated.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [7]:
# Fix ids
falseData1 = fixIds(falseData[:10])
falseData2 = fixIds(falseData[10:22])

# Transform data to json string
evalData1 = falseData1.to_json(orient="records")
evalData2 = falseData2.to_json(orient="records")

# Write as json to file
evaluation1 = open("evaluation1.json", "w")
evaluation1.write(json.dumps(json.loads(evalData1), indent=4, sort_keys=True))
evaluation1.close()

evaluation2 = open("evaluation2.json", "w")
evaluation2.write(json.dumps(json.loads(evalData2), indent=4, sort_keys=True))
evaluation2.close()